In [1]:
import io
import re
import torch
import numpy as np
import pandas as pd

from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset, DataLoader

from google.colab import files

from imblearn.over_sampling import RandomOverSampler

from nltk.tokenize import RegexpTokenizer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from transformers import (AutoTokenizer, AutoModel,
                          AutoModelForSequenceClassification, Trainer,
                          TrainingArguments, EarlyStoppingCallback, AdamW)

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/train_data.csv')
val_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/val_data.csv')
test_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Report1/test_data.csv')

train_texts = train_df['content']
train_labels = train_df['label']
val_texts = val_df['content']
val_labels = val_df['label']
test_texts = test_df['content']
test_labels = test_df['label']

In [4]:
# Tạo TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=4000)
tfidf_vectorizer.fit(train_texts)  # Fit trên tập huấn luyện

# Chuyển đổi dữ liệu thành TF-IDF vectors
train_tfidf_vectors = tfidf_vectorizer.transform(train_texts).toarray()
val_tfidf_vectors = tfidf_vectorizer.transform(val_texts).toarray()
test_tfidf_vectors = tfidf_vectorizer.transform(test_texts).toarray()

In [5]:
# Tạo tokenizer và model từ PhoBERT
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
phobert_model = AutoModel.from_pretrained("vinai/phobert-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/543M [00:00<?, ?B/s]

In [6]:
# Tokenize dữ liệu với padding
def tokenize_and_pad(texts, tokenizer, max_length=256):
    encodings = tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='pt'
    )
    return encodings

train_encodings = tokenize_and_pad(train_texts.tolist(), tokenizer, max_length=256)
val_encodings = tokenize_and_pad(val_texts.tolist(), tokenizer, max_length=256)
test_encodings = tokenize_and_pad(test_texts.tolist(), tokenizer, max_length=256)

In [7]:
class CombinedDataset(Dataset):
    def __init__(self, encodings, labels, tfidf_vectors):
        self.encodings = encodings
        self.labels = labels
        self.tfidf_vectors = tfidf_vectors

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).long()
        tfidf_vector = torch.tensor(self.tfidf_vectors[idx], dtype=torch.float32)
        return {'input_ids': item['input_ids'], 'attention_mask': item['attention_mask'], 'labels': item['labels'], 'tfidf': tfidf_vector}

    def __len__(self):
        return len(self.labels)

In [8]:
train_dataset = CombinedDataset(train_encodings, train_labels.tolist(), train_tfidf_vectors)
val_dataset = CombinedDataset(val_encodings, val_labels.tolist(), val_tfidf_vectors)
test_dataset = CombinedDataset(test_encodings, test_labels.tolist(), test_tfidf_vectors)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [9]:
class CombinedModel(torch.nn.Module):
    def __init__(self, phobert_model, tfidf_dim):
        super(CombinedModel, self).__init__()
        self.phobert_model = phobert_model
        self.tfidf_dim = tfidf_dim
        self.fc = torch.nn.Linear(self.phobert_model.config.hidden_size + self.tfidf_dim, 2)
        self.loss_fn = CrossEntropyLoss()

    def forward(self, input_ids=None, attention_mask=None, tfidf=None, labels=None):
        # Lấy outputs từ PhoBERT
        outputs = self.phobert_model(input_ids=input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state
        pooled_output = hidden_states[:, 0, :]  # Sử dụng hidden state của token [CLS]

        # Kết hợp TF-IDF với các đặc trưng từ PhoBERT
        combined_features = torch.cat((pooled_output, tfidf), dim=1)
        logits = self.fc(combined_features)

        # Tính toán mất mát nếu nhãn được cung cấp
        loss = None
        if labels is not None:
            loss = self.loss_fn(logits, labels)

        return {"loss": loss, "logits": logits}

In [10]:
# Tạo mô hình
model = CombinedModel(phobert_model, tfidf_dim=train_tfidf_vectors.shape[1])

In [11]:
# Định nghĩa hàm huấn luyện với các siêu tham số có thể tinh chỉnh
def train_model(learning_rate, weight_decay, num_train_epochs):
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=50,
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=learning_rate,
        save_strategy="steps",
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
    )

    trainer.train()

    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

In [12]:
# Tinh chỉnh siêu tham số với Grid Search
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'weight_decay': [0.01, 0.015, 0.02],
    'num_train_epochs': [3, 4, 5],
}

best_params = None
best_score = float('inf')

In [13]:
for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for epochs in param_grid['num_train_epochs']:
            eval_loss = train_model(lr, wd, epochs)
            print(f"Learning rate: {lr}, Weight decay: {wd}, Epochs: {epochs}, Eval loss: {eval_loss}")
            if eval_loss < best_score:
                best_score = eval_loss
                best_params = {'learning_rate': lr, 'weight_decay': wd, 'num_train_epochs': epochs}

print(f"Best params: {best_params}, Best eval loss: {best_score}")

Step,Training Loss,Validation Loss
50,0.662100,0.656606
100,0.578500,0.558509
150,0.412300,0.466176
200,0.509700,0.499199
250,0.371100,0.344510
300,0.359700,0.494841
350,0.198800,0.445512
400,0.371700,0.311431
450,0.245100,0.320305
500,0.351600,0.325018


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 3, Eval loss: 0.31143131852149963


Step,Training Loss,Validation Loss
50,0.240300,0.322921
100,0.217600,0.349706
150,0.186300,0.368319
200,0.143800,0.364252


Learning rate: 5e-05, Weight decay: 0.01, Epochs: 4, Eval loss: 0.32292139530181885


Step,Training Loss,Validation Loss
50,0.232700,0.344495
100,0.213200,0.344123
150,0.198900,0.373789
200,0.110300,0.356163
250,0.165800,0.393378


...

Learning rate: 2e-05, Weight decay: 0.02, Epochs: 3, Eval loss: 0.4158640205860138


Step,Training Loss,Validation Loss
50,0.002600,0.429920
100,0.002500,0.482505
150,0.002200,0.456573
200,0.001900,0.444228


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 4, Eval loss: 0.4299204349517822


Step,Training Loss,Validation Loss
50,0.002500,0.439495
100,0.002400,0.491282
150,0.002200,0.460125
200,0.001900,0.445452


Learning rate: 2e-05, Weight decay: 0.02, Epochs: 5, Eval loss: 0.43949466943740845
Best params: {'learning_rate': 5e-05, 'weight_decay': 0.01, 'num_train_epochs': 3}, Best eval loss: 0.31143131852149963


In [52]:
# Khởi tạo Trainer với Dừng sớm (Early Stopping)
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=best_params['weight_decay'],
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    evaluation_strategy="steps",
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=best_params['learning_rate'],
)

trainer_with_early_stop = Trainer(
    model=model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [78]:
trainer_with_early_stop.train()

Step,Training Loss,Validation Loss
50,0.001300,0.405076
100,0.001200,0.451599
150,0.065800,0.380872
200,0.146200,0.390066
250,0.085700,0.419602
300,0.251100,0.588869
350,0.061200,0.563959


TrainOutput(global_step=350, training_loss=0.05151999912623848, metrics={'train_runtime': 271.2146, 'train_samples_per_second': 22.134, 'train_steps_per_second': 2.776, 'total_flos': 0.0, 'train_loss': 0.05151999912623848, 'epoch': 1.3944223107569722})

In [79]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

In [80]:
print(pred_labels2[:])  # In ra dự đoán

[0 1 0 1 1 1 0 1 1 1 1 0 0 1 1 0 0 0 0 1 0 1 0 1 0 1 1 0 1 1 1 0 1 1 0 0 1
 0 0 0 1 1 1 0 0 1 0 0 1 1 0 0 0 0 0 0 1 0 0 1 0 1 0 1 1 0 0 0 0 1 1 1 0 0
 1 1 0 0 1 1 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 1 1 0 1 0 0 0 0 1 1 0 0 1 1
 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 1 0 0 1 1 1 1 1
 1 1 1 1 0 0 0 0 1 1 1 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 1 1 1 1 1 0 1 1 0 1 1
 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 1 0 0 1 1 0
 1 0 0 1 1 1 0 1 1 0 0 1 0 0 0 0 0 1 1 0 1 1 0 1 0 1 0 1 1 0 1 1 0 0 0 0 1
 1 0 0 0 1 1 0 1 0 1 1 0 0 0 1 1 0 0 1 1 1 0 1 1 0 0 1 0 1 0 0 1 1 1 1 0 0
 0 1 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0 1 0 1 0 1 0 1 0 0 1 0 1 1 1 1 0 1 1 1
 1 0 1 0 0 1 1 1 0 0 1 0 1 0 1 0 1 1 1 0 1 1 0 0 1 1 1 0 0 0 1 1 1 1 0 1 0
 1 0 0 0 1 0 0 0 0 1 1 1 1 1 1 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0
 0 0 1 0 1 0 0 0 1 1 0 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 1 1 0 1 1 1 0 1
 1 0 1 0 1 0 1 1 0 1 0 0 1 1 1 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 1 1 1 0 1 

In [81]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')

Accuracy: 0.942116
Precision: 0.937984
Recall: 0.949020
F1 Score: 0.943470
AUC: 0.975387


In [82]:
# Dự đoán nhãn cho tập kiểm tra
predictions3 = trainer_with_early_stop.predict(test_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels3 = np.argmax(predictions3.predictions, axis=1)

In [83]:
print(pred_labels3[:])  # In ra dự đoán

[0 0 1 0 0 0 0 1 1 0 0]
